# DREAM thumbnails

This notebook generates the thumbnails used in the DREAM user guide.

In [ ]:
import scipp as sc
from ess import dream, powder
import ess.dream.data  # noqa: F401
from ess.powder.types import *
import matplotlib.pyplot as plt
import pandas as pd
import plopp as pp

In [ ]:
workflow = dream.DreamGeant4Workflow(run_norm=powder.RunNormalization.proton_charge)

In [ ]:
workflow[Filename[SampleRun]] = dream.data.simulated_diamond_sample()
workflow[Filename[VanadiumRun]] = dream.data.simulated_vanadium_sample()
workflow[Filename[BackgroundRun]] = dream.data.simulated_empty_can()
workflow[CalibrationFilename] = None

workflow[MonitorFilename[SampleRun]] = dream.data.simulated_monitor_diamond_sample()
workflow[MonitorFilename[VanadiumRun]] = dream.data.simulated_monitor_vanadium_sample()
workflow[MonitorFilename[BackgroundRun]] = dream.data.simulated_monitor_empty_can()
workflow[CaveMonitorPosition] = sc.vector([0.0, 0.0, -4220.0], unit="mm")

workflow[dream.InstrumentConfiguration] = dream.InstrumentConfiguration.high_flux
# Select a detector bank:
workflow[NeXusDetectorName] = "mantle"
# We drop uncertainties where they would otherwise lead to correlations:
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
# Edges for binning in d-spacing:
workflow[DspacingBins] = sc.linspace("dspacing", 0.3, 2.3434, 201, unit="angstrom")

# Do not mask any pixels / voxels:
workflow = powder.with_pixel_mask_filenames(workflow, [])

## Basic powder workflow

In [ ]:
results = workflow.compute([IofTof, ReducedTofCIF])
result = results[IofTof]
cif_data = results[ReducedTofCIF]

In [ ]:
histogram = result.hist()

In [ ]:
def basic_powder_plot(style: str):
    with plt.style.context(style):
        fig, ax = plt.subplots(layout='constrained', figsize=(3, 2.5))
        _ = histogram.plot(ax=ax)
        ax.set_xlim((10000, 65000))
        ax.set_xlabel(r'$t$ [µs]')
        ax.set_ylabel(r'$I(t)$')
    return fig

In [ ]:
fig = basic_powder_plot('default')
fig.savefig(
    "../../docs/_static/thumbnails/dream_basic_powder_reduction_light.svg",
    transparent=True,
)
fig

In [ ]:
fig = basic_powder_plot('dark_background')
fig.savefig(
    "../../docs/_static/thumbnails/dream_basic_powder_reduction_dark.svg",
    transparent=True,
)
fig

## Advanced powder workflow

In [ ]:
detector_names = ["mantle", "endcap_forward", "endcap_backward", "high_resolution"]
two_theta_bins = [
    sc.linspace(dim="two_theta", unit="rad", start=0.77, stop=2.36, num=70),
    sc.linspace(dim="two_theta", unit="rad", start=0.24, stop=0.71, num=30),
    sc.linspace(dim="two_theta", unit="rad", start=2.42, stop=2.91, num=50),
    sc.linspace(dim="two_theta", unit="rad", start=2.91, stop=3.11, num=10),
]
parameter_table = pd.DataFrame(
    {NeXusDetectorName: detector_names,
     TwoThetaBins: two_theta_bins,
     },
    index=detector_names
).rename_axis(index='detector')

all_detector_workflow = workflow.copy()
mapped = all_detector_workflow[IofDspacingTwoTheta].map(parameter_table)
all_detector_workflow[IofDspacingTwoTheta] = mapped.reduce(func=powder.grouping.collect_detectors)

result = all_detector_workflow.compute(IofDspacingTwoTheta)

In [ ]:
histogram = result.bin(dspacing=80).hist()

In [ ]:
def advanced_powder_plot(style: str):
    with plt.style.context(style):
        fig, ax = plt.subplots(layout='constrained', figsize=(3, 2.5))
        pf = pp.imagefigure(*(pp.Node(da) for da in histogram.values()), norm='log', cbar=True, ax=ax)
        pf.view.colormapper.ylabel = None
        ax.set_xlabel(r'$d$ [Å]')
        ax.set_ylabel(r'$2\theta$ [rad]')
    return fig

In [ ]:
fig = advanced_powder_plot('default')
fig.savefig(
    "../../docs/_static/thumbnails/dream_advanced_powder_reduction_light.svg",
    transparent=True,
)
fig

In [ ]:
fig = advanced_powder_plot('dark_background')
fig.savefig(
    "../../docs/_static/thumbnails/dream_advanced_powder_reduction_dark.svg",
    transparent=True,
)
fig